# Read API data in PySpark
I am using a function in order to be able to focus on one part of the data.

This approach comes from a couple interesting article about parallizing the function, to use the facilities of Spark more effectively.
- [How to Execute a REST API call on Apache Spark the Right Way | by James S Hocking | Geek Culture | Aug, 2021 | Medium | Geek Culture](https://medium.com/geekculture/how-to-execute-a-rest-api-call-on-apache-spark-the-right-way-in-python-4367f2740e78)
- [REST API to Spark Dataframe (datanoon.com)](https://datanoon.com/blog/loading_data_rest_api_to_spark/)


## Need to import libraries
First you will need to make sure that all of the relevant Python libraries are load.  I have included a couple that I am not going to use.  I used the two sources above for my reference and removed the imports that I know I won't be using and left the ones that I might need.

In [1]:
import requests
import json
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
from pyspark.sql import Row

StatementMeta(Main, 18, 1, Finished, Available)

In [2]:
from notebookutils import mssparkutils

StatementMeta(Main, 18, 2, Finished, Available)

## Create a function to read the data
- verb is the kind of call that is made:  GET, POST
- url is the web address that you pass in
- headers are the name value pairs that you pass in
- body is the body that you pass in with a post.



In [3]:
def executeRestApi(verb, url, headers, body):
  # Initialize the results.  This will return a 
  res = None
  # Make API request, get response object back, create dataframe from above schema.
  try:
    if verb == "get":
      res = requests.get(url, data=body, headers=headers)
    else:
      res = requests.post(url, data=body, headers=headers)
  except Exception as e:
    return e
  if res != None and res.status_code == 200:
    return json.loads(res.text)
  return None

StatementMeta(Main, 18, 3, Finished, Available)

## PUMS Data API
This notebook explores the PUMS datasource ([American Community Survey Microdata (census.gov)](https://www.census.gov/programs-surveys/acs/microdata.html).  This describes the APIs for microdata https://www.census.gov/data/developers/data-sets/census-microdata-api.html.

### Call the code
The queryURL is simply the URL that we will be passing in.  The parts of the URL are as follows:
- https://api.census.gov: This is the root of the census APIs.  I assume that I could experiment with changing the path
/data/2021/acs/acs1/pums:  This is the path.  It is easy to guess what this is about from the name
    - 2021 is the year
    - acs/acs1/pums:  this is the set (the acs1 refers to the one year data as opposed to acs5 which is 5 year data)
- ?get=SEX,PWGTP,MAR':  these are the arguments.  "get" is listing the columns you want

In [4]:
PUMSKey = mssparkutils.credentials.getSecret("loski-202303","MicrodataKey")
queryURL = 'https://api.census.gov/data/2021/acs/acs1/pums?get=WGTP,FHICOVP&HHLDRHISP=02:99&key=' + PUMSKey
headers = {
    'content-type': "application/json"
}
body = json.dumps({
})

results = executeRestApi("get", queryURL, headers, body)

StatementMeta(Main, 18, 4, Finished, Available)

### View the results
The function above will convert the text to a json document.  There are some approaches that you can use to simplify the output so that it is easier to work with.  They are referenced in the articles above.  For this session, I will be using brute force to determine what is in the results and will then craft code to load the data into a dataframe.

The results are an array of arrays of strings.  The square brackets \[\] bound each array. There are arrays of strings, which are themselves items in an array.

In [5]:
results 

StatementMeta(Main, 18, 5, Finished, Available)

[['WGTP', 'FHICOVP', 'HHLDRHISP'],
 ['283', '0', '3'],
 ['31', '0', '2'],
 ['31', '0', '2'],
 ['31', '0', '2'],
 ['82', '0', '2'],
 ['82', '0', '2'],
 ['170', '0', '2'],
 ['170', '1', '2'],
 ['170', '1', '2'],
 ['170', '1', '2'],
 ['170', '1', '2'],
 ['137', '0', '2'],
 ['137', '0', '2'],
 ['137', '0', '2'],
 ['137', '0', '2'],
 ['137', '0', '2'],
 ['484', '0', '3'],
 ['115', '0', '11'],
 ['115', '0', '11'],
 ['108', '0', '2'],
 ['108', '0', '2'],
 ['108', '0', '2'],
 ['108', '0', '2'],
 ['108', '0', '2'],
 ['35', '0', '3'],
 ['35', '1', '3'],
 ['35', '0', '3'],
 ['35', '0', '3'],
 ['35', '0', '3'],
 ['114', '0', '9'],
 ['114', '0', '9'],
 ['49', '0', '2'],
 ['49', '1', '2'],
 ['265', '0', '21'],
 ['265', '0', '21'],
 ['265', '0', '21'],
 ['73', '0', '2'],
 ['73', '1', '2'],
 ['81', '0', '2'],
 ['136', '0', '2'],
 ['136', '0', '2'],
 ['100', '0', '2'],
 ['100', '0', '2'],
 ['100', '0', '2'],
 ['100', '0', '2'],
 ['183', '0', '11'],
 ['183', '0', '11'],
 ['56', '1', '2'],
 ['56', '1', '

### Parse the results
I want to create a dataframe from the results.  There are two pieces to this:  the header and data.  The first row is the header.  The rest is data.

Getting data from arrays is very easy.  I just need to take my variable (results) and reference the rows.  results\[0\] will give me the header row (0 is the first item in an array), results\[1:\] will get the rows from the second row to the end (1 is the second row; colon with no value following it will go to the end of the array).  Note that if I have one value, it returns the first row as an array.  With the range, it returns an array of arrays.

In [27]:
results[0]

StatementMeta(Main, 10, 14, Finished, Available)

['WGTP', 'FHICOVP', 'HHLDRHISP']

In [28]:
results[1:]

StatementMeta(Main, 10, 15, Finished, Available)

[['283', '0', '3'],
 ['31', '0', '2'],
 ['31', '0', '2'],
 ['31', '0', '2'],
 ['82', '0', '2'],
 ['82', '0', '2'],
 ['170', '0', '2'],
 ['170', '1', '2'],
 ['170', '1', '2'],
 ['170', '1', '2'],
 ['170', '1', '2'],
 ['137', '0', '2'],
 ['137', '0', '2'],
 ['137', '0', '2'],
 ['137', '0', '2'],
 ['137', '0', '2'],
 ['484', '0', '3'],
 ['115', '0', '11'],
 ['115', '0', '11'],
 ['108', '0', '2'],
 ['108', '0', '2'],
 ['108', '0', '2'],
 ['108', '0', '2'],
 ['108', '0', '2'],
 ['35', '0', '3'],
 ['35', '1', '3'],
 ['35', '0', '3'],
 ['35', '0', '3'],
 ['35', '0', '3'],
 ['114', '0', '9'],
 ['114', '0', '9'],
 ['49', '0', '2'],
 ['49', '1', '2'],
 ['265', '0', '21'],
 ['265', '0', '21'],
 ['265', '0', '21'],
 ['73', '0', '2'],
 ['73', '1', '2'],
 ['81', '0', '2'],
 ['136', '0', '2'],
 ['136', '0', '2'],
 ['100', '0', '2'],
 ['100', '0', '2'],
 ['100', '0', '2'],
 ['100', '0', '2'],
 ['183', '0', '11'],
 ['183', '0', '11'],
 ['56', '1', '2'],
 ['56', '1', '2'],
 ['56', '0', '2'],
 ['56', '0'

### Create a dataframe from the results
What I can do next is to create a dataframe.  Dataframes are resultsets that you can work with.  Once we have the data in a dataframe, we can manipulate it further:  summarizes, filtering, even writing it to disk.

There are different ways to create a dataframe.  In this case, I pass in the data as an array of arrays as data and array of strings as the column header.


In [6]:
df = spark.createDataFrame(results[1:], results[0])
df.show()

StatementMeta(Main, 18, 6, Finished, Available)

+----+-------+---------+
|WGTP|FHICOVP|HHLDRHISP|
+----+-------+---------+
| 283|      0|        3|
|  31|      0|        2|
|  31|      0|        2|
|  31|      0|        2|
|  82|      0|        2|
|  82|      0|        2|
| 170|      0|        2|
| 170|      1|        2|
| 170|      1|        2|
| 170|      1|        2|
| 170|      1|        2|
| 137|      0|        2|
| 137|      0|        2|
| 137|      0|        2|
| 137|      0|        2|
| 137|      0|        2|
| 484|      0|        3|
| 115|      0|       11|
| 115|      0|       11|
| 108|      0|        2|
+----+-------+---------+
only showing top 20 rows



In [7]:
display(df)

StatementMeta(Main, 18, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5a63b32a-af9d-4cd1-93c3-4b9940dcaebd)

### Run basic group by query


In [46]:
df.printSchema()

StatementMeta(Main, 10, 33, Finished, Available)

root
 |-- WGTP: string (nullable = true)
 |-- FHICOVP: string (nullable = true)
 |-- HHLDRHISP: string (nullable = true)



In [48]:
df2 = df.withColumn("WEIGHT", col("WGTP").cast("int"))

StatementMeta(Main, 10, 35, Finished, Available)

In [51]:
df2.groupBy("FHICOVP","HHLDRHISP").sum("WEIGHT").show()

StatementMeta(Main, 10, 38, Finished, Available)

+-------+---------+-----------+
|FHICOVP|HHLDRHISP|sum(WEIGHT)|
+-------+---------+-----------+
|      0|        6|     124816|
|      0|        4|    1840376|
|      1|        4|     456848|
|      1|       11|     517466|
|      0|       19|     542239|
|      0|        5|    1723195|
|      0|       22|      34737|
|      1|       15|      25226|
|      1|       19|     143060|
|      0|       20|      47834|
|      0|       24|    1675994|
|      1|        2|    6534539|
|      1|       17|     169234|
|      0|       18|      18637|
|      0|       13|     235642|
|      1|       20|      12465|
|      1|        5|     482882|
|      0|       10|     179540|
|      0|       11|    1943507|
|      0|       15|     138417|
|      1|       16|     268721|
|      1|       10|      46488|
|      1|        3|    1086958|
|      0|        3|    4072744|
|      1|       23|     167014|
|      1|       14|      18960|
|      1|       12|       5203|
|      1|        7|     297244|
|      0

## PUMS metadata
The Census Bureau has included some metadata to describe the columns, providing full description and value lists.

In [8]:
queryURL = 'https://api.census.gov/data/2021/acs/acs1/pums/variables.json'
headers = {
    'content-type': "application/json"
}
body = json.dumps({
})

results = executeRestApi("get", queryURL, headers, body)

StatementMeta(Main, 18, 8, Finished, Available)

### View the results
The structure of this is very different. The whole is a single object with one property "variables." variables is a dictionary, where the keys are the values that are used in the URL or in the get clause.

In [9]:
results

StatementMeta(Main, 18, 9, Finished, Available)

{'variables': {'for': {'label': "Census API FIPS 'for' clause",
   'concept': 'Census API Geography Specification',
   'predicateType': 'fips-for',
   'group': 'N/A',
   'limit': 0,
   'predicateOnly': True},
  'in': {'label': "Census API FIPS 'in' clause",
   'concept': 'Census API Geography Specification',
   'predicateType': 'fips-in',
   'group': 'N/A',
   'limit': 0,
   'predicateOnly': True},
  'ucgid': {'label': 'Uniform Census Geography Identifier clause',
   'concept': 'Census API Geography Specification',
   'predicateType': 'ucgid',
   'group': 'N/A',
   'limit': 0,
   'predicateOnly': True},
  'HHLANP': {'label': 'Detailed household language',
   'predicateType': 'string',
   'group': 'N/A',
   'limit': 0,
   'suggested-weight': 'WGTP',
   'values': {'item': {'1210': 'Portuguese',
     '1350': 'Hindi',
     '2560': 'Japanese',
     '5940': 'Fulah',
     '1765': 'Tamil',
     '1263': 'Slovak',
     '1134': 'Afrikaans',
     '1025': 'Other English-based Creole languages',
   

### Find the keys
It is easy to find the keys.  Running the following shows me that that is one key (variables).

I can get the keys for the variables item.

In [10]:
results.keys()

StatementMeta(Main, 18, 10, Finished, Available)

dict_keys(['variables'])

In [10]:
results["variables"].keys()

StatementMeta(Main, 17, 10, Finished, Available)

dict_keys(['for', 'in', 'ucgid', 'HHLANP', 'FBATHP', 'DRIVESP', 'WGTP23', 'WGTP22', 'WGTP25', 'WGTP24', 'RACNH', 'WGTP21', 'FWATP', 'WGTP20', 'WGTP27', 'WGTP26', 'WGTP29', 'WGTP28', 'FBROADBNDP', 'FDRATXP', 'FWKWNP', 'HOTWAT', 'FWKHP', 'FFULP', 'WORKSTAT', 'FRACP', 'FJWDP', 'WGTP34', 'WGTP33', 'WGTP36', 'PINCP', 'WGTP35', 'FPOBP', 'WGTP30', 'WGTP32', 'STOV', 'FMHP', 'WGTP31', 'RACAIAN', 'WGTP38', 'WGTP37', 'WGTP39', 'PUBCOV', 'SRNT', 'SEX', 'WGTP45', 'WGTP44', 'WGTP47', 'DOUT', 'FACCESSP', 'WGTP46', 'WGTP41', 'WGTP40', 'OTHSVCEX', 'WGTP43', 'WGTP42', 'RACPI', 'INDP', 'WGTP49', 'WGTP48', 'PRIVCOV', 'SFN', 'FINTP', 'HUPAC', 'SFR', 'WGTP50', 'FBLDP', 'WGTP56', 'WGTP55', 'WGTP58', 'WGTP57', 'WGTP52', 'WGTP51', 'DEAR', 'WGTP54', 'DIS', 'WGTP53', 'ACR', 'VACS', 'FINSP', 'WGTP59', 'FMILPP', 'MARHYP', 'ADJHSG', 'PAP', 'WGTP7', 'PWGTP30', 'WGTP6', 'PWGTP31', 'WGTP5', 'HINCP', 'PWGTP32', 'WKWN', 'WGTP4', 'PWGTP33', 'PWGTP34', 'PWGTP35', 'WGTP9', 'PWGTP36', 'WGTP8', 'WGTP3', 'WGTP2', 'RACWHT', 'W

In [11]:
results["variables"]["HHLDRHISP"]["values"]["item"]

StatementMeta(Main, 18, 11, Finished, Available)

{'24': 'All Other Spanish/Hispanic/Latino',
 '04': 'Cuban',
 '06': 'Costa Rican',
 '07': 'Guatemalan',
 '02': 'Mexican',
 '10': 'Panamanian',
 '15': 'Chilean',
 '05': 'Dominican',
 '09': 'Nicaraguan',
 '14': 'Bolivian',
 '19': 'Peruvian',
 '21': 'Venezuelan',
 '01': 'Not Spanish/Hispanic/Latino',
 '11': 'Salvadoran',
 '20': 'Uruguayan',
 '22': 'Other South American',
 '12': 'Other Central American',
 '13': 'Argentinean',
 '16': 'Colombian',
 '17': 'Ecuadorian',
 '18': 'Paraguayan',
 '0': 'N/A (GQ/vacant)',
 '23': 'Spaniard',
 '03': 'Puerto Rican',
 '08': 'Honduran'}

In [12]:
# https://sparkbyexamples.com/pyspark/pyspark-create-dataframe-from-dictionary/
from pyspark.sql.types import StructField, StructType, StringType, MapType
results["variables"]["HHLANP"]["values"]
dfPUMS = spark.createDataFrame(data = [results["variables"]["HHLANP"]["values"]]  )

StatementMeta(Main, 18, 12, Finished, Available)

In [13]:
dfPUMS.printSchema()

StatementMeta(Main, 18, 13, Finished, Available)

root
 |-- item: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [17]:
# https://github.com/spark-examples/pyspark-examples/blob/master/pyspark-maptype-dataframe-column.py

from pyspark.sql.functions import explode
dfMetaData  = dfPUMS.select(explode(dfPUMS.item))
dfMetaData.printSchema()

StatementMeta(Main, 18, 17, Finished, Available)

root
 |-- key: string (nullable = false)
 |-- value: string (nullable = true)



In [18]:
dfMetaData.show()

StatementMeta(Main, 18, 18, Finished, Available)

+----+--------------------+
| key|               value|
+----+--------------------+
|6230|       Gbe languages|
|9500|English only hous...|
|2430|                Thai|
|1220|            Romanian|
|1582|           Hungarian|
|1340|        India N.E.C.|
|2030|     Min Nan Chinese|
|2270|      Chin languages|
|1730|              Telugu|
|1970|             Chinese|
|1210|          Portuguese|
|4565|Chaldean Neo-Aramaic|
|4840|              Somali|
|1737|             Kannada|
|6120|    Akan (incl. Twi)|
|5950|               Wolof|
|1110|              German|
|1231|               Irish|
|2560|            Japanese|
|1350|               Hindi|
+----+--------------------+
only showing top 20 rows



## CPS Basic Monthly
This is the API used to query the Current Population Survey Basic Monthly data:  [CPS Basic Monthly (census.gov)
](https://www.census.gov/data/developers/data-sets/census-microdata-api/cps/basic.html)

The basic URL is https://api.census.gov/data/{year}/cps/basic/{month}

The rest of the URL formatting is the same as PUMS.

### Use case
This is a job

In [74]:
queryURL = 'https://api.census.gov/data/2023/cps/basic/jan?get=PWSSWGT,PEHSPNON,PTIO2OCD&PTIO1OCD=1065'
headers = {
    'content-type': "application/json"
}
body = json.dumps({
})

results = executeRestApi("get", queryURL, headers, body)

StatementMeta(Main, 10, 61, Finished, Available)

In [76]:
df = spark.createDataFrame(results[1:], results[0])

StatementMeta(Main, 10, 63, Finished, Available)

In [77]:
 df.show()

StatementMeta(Main, 10, 64, Finished, Available)

+---------+--------+--------+--------+--------+
|  PWSSWGT|PEHSPNON|PTIO1OCD|PTIO2OCD|PTIO1OCD|
+---------+--------+--------+--------+--------+
|4291.4519|       2|    1065|      -1|    1065|
|2458.0463|       2|    1065|      -1|    1065|
|4041.1388|       2|    1065|      -1|    1065|
| 736.2925|       2|    1065|      -1|    1065|
|3829.3150|       2|    1065|      -1|    1065|
|3064.9641|       2|    1065|      -1|    1065|
|2914.9950|       2|    1065|      -1|    1065|
|2168.3438|       2|    1065|      -1|    1065|
| 547.5260|       2|    1065|      -1|    1065|
|4802.3933|       2|    1065|      -1|    1065|
|4876.6889|       2|    1065|      -1|    1065|
| 870.7179|       2|    1065|      -1|    1065|
|2980.3243|       2|    1065|      -1|    1065|
|3743.4214|       2|    1065|      -1|    1065|
|3141.1348|       2|    1065|      -1|    1065|
|4530.3208|       2|    1065|      -1|    1065|
|1125.8354|       2|    1065|      -1|    1065|
| 415.1696|       2|    1065|      -1|  

## Bureau of Labor Statistics reports
This is based on some example code from the BLS web site.  It demonstrates how to use the post command.

[Accessing the Public Data API with Python : U.S. Bureau of Labor Statistics (bls.gov)](https://www.bls.gov/developers/api_python.htm)

In [78]:
queryURL = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
headers = {
    'content-type': "application/json"
}
body = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2014"})

results = executeRestApi("post", queryURL, headers, body)

StatementMeta(Main, 10, 65, Finished, Available)

In [82]:
results

StatementMeta(Main, 10, 69, Finished, Available)

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 226,
 'message': [],
 'Results': {'series': [{'seriesID': 'CUUR0000SA0',
    'data': [{'year': '2014',
      'period': 'M12',
      'periodName': 'December',
      'value': '234.812',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M11',
      'periodName': 'November',
      'value': '236.151',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M10',
      'periodName': 'October',
      'value': '237.433',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M09',
      'periodName': 'September',
      'value': '238.031',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M08',
      'periodName': 'August',
      'value': '237.852',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M07',
      'periodName': 'July',
      'value': '238.250',
      'footnotes': [{}]},
     {'year': '2014',
      'period': 'M06',
      'periodName': 'June',
      'value': '238.343',
   

### Parse the json
If you know how to read this, you begin to parse this.  It is JSON.

At the top, you have one object (between curly braces {}).  That object has properties, which PySpark treats as a dictionary.  To get the value, just have take the results and ask for the property name.

In [86]:
print(results["status"])
print(results["responseTime"])


StatementMeta(Main, 10, 73, Finished, Available)

REQUEST_SUCCEEDED
226


### Create dataframe
The Results element is another dictionary, with a property (series) that is an array of dictionary structures.  createDateFrame works very nicely with that array.

In [87]:
df = spark.createDataFrame(results["Results"]["series"] )

StatementMeta(Main, 10, 74, Finished, Available)

In [88]:
display(df)

StatementMeta(Main, 10, 75, Finished, Available)

SynapseWidget(Synapse.DataFrame, accfac01-c0b6-4ce1-8b42-1161f847c96c)

## GSA Regulations
This is used to get information about regulations.  Once you have the information, you can submit comments using this API.

https://open.gsa.gov/api/regulationsgov/

In [15]:
GSAKey = mssparkutils.credentials.getSecret("Loski-202303", "GSAKey")
queryURL = 'https://api.regulations.gov/v4/documents?filter[searchTerm]=water&api_key=' + GSAKey
headers = {
    'content-type': "application/json" 
}
body = json.dumps({})

results = executeRestApi("get", queryURL, headers, body)

StatementMeta(Main, 13, 15, Finished, Available)

In [16]:
results

StatementMeta(Main, 13, 16, Finished, Available)

## CMS Marketplace API

In [9]:
CMSKey =  mssparkutils.credentials.getSecret('Loski-202303','CMSKey')


queryURL = 'https://marketplace.api.healthcare.gov/api/v1/counties/by/zip/76051?apikey=' + CMSKey
headers = {
    'content-type': "application/json"
}
body = json.dumps({})

results = executeRestApi("post", queryURL, headers, body)

StatementMeta(Main, 14, 9, Finished, Available)

In [10]:
results

StatementMeta(Main, 14, 10, Finished, Available)

In [19]:
CMSKey =  mssparkutils.credentials.getSecret('Loski-202303','CMSKey')

queryURL = 'https://marketplace.api.healthcare.gov/api/v1/plans/search?apikey=' + CMSKey
headers = {
    'content-type': "application/json"
}
body = json.dumps(
     {
    "household": {
      "income": 52000,
      "people": [
        {
          "age": 27,
          "aptc_eligible": True,
          "gender": "Female",
          "uses_tobacco": False
        }
      ]
    },
    "market": "Individual",
    "place": {
      "countyfips": "22033",
      "state": "LA",
      "zipcode": "70810"
    },
    "year": 2019
}

)

results = executeRestApi("post", queryURL, headers, body)

StatementMeta(Main, 18, 19, Finished, Available)

In [20]:
 results

StatementMeta(Main, 18, 20, Finished, Available)

{'plans': [{'id': '19636LA0230006',
   'name': 'Community Blue 70/50 $4500',
   'premium': 263.91,
   'premium_w_credit': 263.91,
   'ehb_premium': 263.91,
   'pediatric_ehb_premium': 0,
   'aptc_eligible_premium': 263.91,
   'metal_level': 'Bronze',
   'type': 'POS',
   'state': 'LA',
   'benefits': [{'name': 'Primary Care Visit to Treat an Injury or Illness',
     'covered': True,
     'cost_sharings': [{'coinsurance_rate': 0.3,
       'coinsurance_options': 'Coinsurance after deductible',
       'copay_amount': 0,
       'copay_options': 'Not Applicable',
       'network_tier': 'In-Network',
       'csr': 'Exchange variant (no CSR)',
       'display_string': '30% Coinsurance after deductible'},
      {'coinsurance_rate': 0.5,
       'coinsurance_options': 'Coinsurance after deductible',
       'copay_amount': 0,
       'copay_options': 'Not Applicable',
       'network_tier': 'Out-of-Network',
       'csr': 'Exchange variant (no CSR)',
       'display_string': '50% Coinsurance after

In [21]:
df = spark.createDataFrame(results["plans"])

StatementMeta(Main, 18, 21, Finished, Available)

In [22]:
display(df)

StatementMeta(Main, 18, 22, Finished, Available)

SynapseWidget(Synapse.DataFrame, 301d6f9e-4cc4-4cdb-8390-7758deb28c97)